In [1]:
import os
import pandas as pd
import warnings
import numpy as np

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/datasets")
os.chdir("C:/diego_tools/labo3/dataset")
####################################################

In [3]:
periodo_test=201902

In [4]:
arch_predicciones = "emp3_lightgbm_prediccion.csv"
arch_prod_ids_prediccion = "productos_a_predecir.csv"
arch_sellout_train = "emp3_sellout_product_train.csv"

In [5]:
def error_rate(y, y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)
    
    #Las predicciones negativas se convierten a 0
    y_pred = np.maximum(y_pred, 0)
    
    dif_abs = sum(abs(y - y_pred))
    suma_real = sum(y)
    
    return round(100*dif_abs/suma_real,2)

In [6]:
df_sellout = pd.read_csv(arch_sellout_train)

In [7]:
df_sellout.head()

,product_id,periodo,imputado,tn,cust_request_tn,cust_request_qty,plan_precios_cuidados,periodo_fecha,mes,meses_historia,cat1,cat2,cat3,brand,sku_size,producto_estrella,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe,dif_cust_request_tn,dif_cust_request_tn_porc,tn_lag_1,tn_lag_2,tn_lag_3,tn_lag_4,tn_lag_5,tn_lag_6,tn_lag_7,tn_lag_8,tn_lag_9,tn_lag_10,tn_lag_11,tn_lag_12,tn_lag_13,tn_lag_14,tn_lag_15,tn_mas_2,tn_delta_1,tn_delta_2,tn_delta_3,tn_delta_4,tn_delta_5,tn_delta_6,tn_delta_7,tn_delta_8,tn_delta_9,tn_delta_10,tn_delta_11,tn_delta_12,tn_delta_13,tn_delta_14,tn_delta_15,cat1_tn,cat1_tn_lag_1,cat1_tn_delta_1,cat1_tn_lag_2,cat1_tn_delta_2,cat1_tn_lag_3,cat1_tn_delta_3,cat1_tn_lag_4,cat1_tn_delta_4,cat1_tn_lag_5,cat1_tn_delta_5,cat1_tn_lag_6,cat1_tn_delta_6,cat1_tn_lag_7,cat1_tn_delta_7,cat1_tn_lag_8,cat1_tn_delta_8,cat1_tn_lag_9,cat1_tn_delta_9,cat1_tn_lag_10,cat1_tn_delta_10,cat1_tn_lag_11,cat1_tn_delta_11,cat1_tn_lag_12,cat1_tn_delta_12,cat1_tn_lag_13,cat1_tn_delta_13,cat1_tn_lag_14,cat1_tn_delta_14,cat1_tn_lag_15,cat1_tn_delta_15,cat2_tn,cat2_tn_lag_1,cat2_tn_delta_1,cat2_tn_lag_2,cat2_tn_delta_2,cat2_tn_lag_3,cat2_tn_delta_3,cat2_tn_lag_4,cat2_tn_delta_4,cat2_tn_lag_5,cat2_tn_delta_5,cat2_tn_lag_6,cat2_tn_delta_6,cat2_tn_lag_7,cat2_tn_delta_7,cat2_tn_lag_8,cat2_tn_delta_8,cat2_tn_lag_9,cat2_tn_delta_9,cat2_tn_lag_10,cat2_tn_delta_10,cat2_tn_lag_11,cat2_tn_delta_11,cat2_tn_lag_12,cat2_tn_delta_12,cat2_tn_lag_13,cat2_tn_delta_13,cat2_tn_lag_14,cat2_tn_delta_14,cat2_tn_lag_15,cat2_tn_delta_15,cat3_tn,cat3_tn_lag_1,cat3_tn_delta_1,cat3_tn_lag_2,cat3_tn_delta_2,cat3_tn_lag_3,cat3_tn_delta_3,cat3_tn_lag_4,cat3_tn_delta_4,cat3_tn_lag_5,cat3_tn_delta_5,cat3_tn_lag_6,cat3_tn_delta_6,cat3_tn_lag_7,cat3_tn_delta_7,cat3_tn_lag_8,cat3_tn_delta_8,cat3_tn_lag_9,cat3_tn_delta_9,cat3_tn_lag_10,cat3_tn_delta_10,cat3_tn_lag_11,cat3_tn_delta_11,cat3_tn_lag_12,cat3_tn_delta_12,cat3_tn_lag_13,cat3_tn_delta_13,cat3_tn_lag_14,cat3_tn_delta_14,cat3_tn_lag_15,cat3_tn_delta_15
0,20001,201701,0,934.77,937.73,479.00,0.00,2017-01-01,1,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.00,1.00,25.65,30.40,20.90,1.60,15.91,False,2.95,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1303.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20304.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11153.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3871.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20002,201701,0,550.16,555.19,391.00,0.00,2017-01-01,1,0,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.00,1.00,25.65,30.40,20.90,1.60,15.91,False,5.03,0.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,834.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20304.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11153.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3871.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20009,201701,0,378.08,380.53,429.00,0.00,2017-01-01,1,0,HC,ROPA LAVADO,Liquido,ROPEX1,3000.00,1.00,25.65,30.40,20.90,1.60,15.91,False,2.45,0.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,456.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20304.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11153.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3871.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [8]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23173 entries, 0 to 23172
Columns: 148 entries, product_id to cat3_tn_delta_15
dtypes: bool(1), float64(137), int64(5), object(5)
memory usage: 26.0+ MB


In [9]:
df_prods_prediccion=pd.read_csv(arch_prod_ids_prediccion)
df_prods_prediccion.head()

,product_id
0,20480
1,20481
2,20482
3,20483
4,20484


In [10]:
df_prods_prediccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   product_id  868 non-null    int64
dtypes: int64(1)
memory usage: 6.9 KB


In [11]:
df_predicciones = pd.read_csv(arch_predicciones)
df_predicciones.head()

,product_id,tn_predicted
0,20001,1202.26
1,20002,1041.92
2,20003,694.43
3,20004,541.49
4,20005,525.47


In [12]:
df_predicciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 868 entries, 0 to 867
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    868 non-null    int64  
 1   tn_predicted  868 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 13.7 KB


In [13]:
# Filtro los productos a predecir
df_sellout_test = df_sellout[(df_sellout.product_id.isin(df_prods_prediccion.product_id)) & (df_sellout.periodo==periodo_test)]
df_sellout_test = df_sellout_test.sort_values(by="product_id", ascending=True)
len(df_sellout_test)

868

In [14]:
y_pred_test = df_predicciones.tn_predicted
y_test = df_sellout_test.tn_mas_2

print("TEST Toneladas: ", sum(y_test), " - Tamaño:",y_test.shape)
print("PRED Toneladas: ", sum(y_pred_test), " - Tamaño:",y_pred_test.shape)


er_test = error_rate(np.array(y_test),np.array(y_pred_test))
er_test

TEST Toneladas:  33763.86541000003  - Tamaño: (868,)
PRED Toneladas:  32371.10940927968  - Tamaño: (868,)


25.75